# **INF161 - Bike Traffic Prediction Project**
### *Ole Kristian Westby | owe009@uib.no | H23*

This project uses data from Statens vegvesen and Geofysisk institutt. The goal is to create a model that can predict the volume (how many) of bikers at a given time over Nygårdsbroen. I'll need to prepare, strip and sort the data so I'm left with the data I deem valuable to perform this task. That's what this Jupyter notebook is for. I'll also be explaining my steps throughout the book. At the end, we'll have some juicy, ready data that we'll use to insert into /ready_data/ ready for the model to work on.

### Let's start by importing some libraries.

In [7]:
import numpy as np
import os

### We'll handle the traffic data first.

In [8]:
dir_weather = "raw_data/weather_data/"
files = [f for f in os.listdir(dir_weather) if f.endswith('.csv')]
print(files)



['Florida_2010-01-01_2011-01-01_1654174747.csv', 'Florida_2011-01-01_2012-01-01_1654174772.csv', 'Florida_2012-01-01_2013-01-01_1654174811.csv', 'Florida_2013-01-01_2014-01-01_1654174853.csv', 'Florida_2014-01-01_2015-01-01_1654174868.csv', 'Florida_2015-01-01_2016-01-01_1654174882.csv', 'Florida_2016-01-01_2017-01-01_1654174902.csv', 'Florida_2017-01-01_2018-01-01_1654174925.csv', 'Florida_2018-01-01_2019-01-01_1654175073.csv', 'Florida_2019-01-01_2020-01-01_1654174955.csv', 'Florida_2020-01-01_2021-01-01_1654174973.csv', 'Florida_2021-01-01_2022-01-01_1654174989.csv', 'Florida_2022-01-01_2023-01-01_1688719054.csv', 'Florida_2023-01-01_2023-07-01_1688719120.csv']
